In [1]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
import pandas as pd
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PoissonRegressor
from sklearn.neural_network import MLPClassifier
import numpy as np

import matplotlib.pyplot as plt

In [2]:
from sklearn import datasets

iris = datasets.load_iris(return_X_y=True,as_frame=True)

In [3]:
X = iris[0]
y = iris[1]

In [4]:
X

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [5]:
y

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: target, Length: 150, dtype: int32

In [6]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y,test_size=.2,random_state=42,shuffle=True)

model = Pipeline(
    steps=[
        ('scaler',MinMaxScaler()),
        ('learner',LogisticRegression())
    ]
).fit(Xtrain,ytrain)

In [7]:
ypred = pd.Series(model.predict(Xtest),index=ytest.index)

In [8]:
confusion_matrix(ytest,ypred)

array([[10,  0,  0],
       [ 0,  8,  1],
       [ 0,  0, 11]], dtype=int64)

In [9]:
Xtrain,ytrain

(     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
 22                 4.6               3.6                1.0               0.2
 15                 5.7               4.4                1.5               0.4
 65                 6.7               3.1                4.4               1.4
 11                 4.8               3.4                1.6               0.2
 42                 4.4               3.2                1.3               0.2
 ..                 ...               ...                ...               ...
 71                 6.1               2.8                4.0               1.3
 106                4.9               2.5                4.5               1.7
 14                 5.8               4.0                1.2               0.2
 92                 5.8               2.6                4.0               1.2
 102                7.1               3.0                5.9               2.1
 
 [120 rows x 4 columns],
 22     0
 15     0
 65  

In [10]:

x_sets = []
targets = []

for c in ytrain.unique():

    subset = Xtrain[ytrain!=c].copy()

    subset_target = pd.Series(np.repeat(c,len(subset)),index=subset.index)

    x_sets.append(subset)
    targets.append(subset_target)


In [11]:
neg_Xtrain = pd.concat(x_sets)
neg_Xtrain['neg'] = 1

Xtrain2 = Xtrain.copy()
Xtrain2['neg'] = 0

Xtrain_new = pd.concat([Xtrain2,neg_Xtrain])

neg_ytrain = pd.concat(targets)

ytrain_new = pd.concat([ytrain,neg_ytrain])

Xtest_new = Xtest.copy()
Xtest_new['neg'] = 0


In [12]:
model = Pipeline(
    steps=[
        ('scaler',MinMaxScaler()),
        ('learner',
         MLPClassifier(hidden_layer_sizes=(15,15,15),activation='relu',
                       alpha=.0001,
                       batch_size=len(Xtest_new),
                       learning_rate_init=1e-2,
                       learning_rate='constant',
                       max_iter=100000,
                       tol=1e-12,
                       early_stopping=False,
                       verbose=True,
                       n_iter_no_change=150
                       )
         )
    ]
).fit(Xtrain_new,ytrain_new)

ypred = pd.Series(model.predict(Xtest_new),index=Xtest_new.index)

Iteration 1, loss = 1.17246711
Iteration 2, loss = 1.06069972
Iteration 3, loss = 1.01921446
Iteration 4, loss = 0.95385734
Iteration 5, loss = 0.89072995
Iteration 6, loss = 0.82512619
Iteration 7, loss = 0.77095319
Iteration 8, loss = 0.70248061
Iteration 9, loss = 0.67204197
Iteration 10, loss = 0.62610376
Iteration 11, loss = 0.61268886
Iteration 12, loss = 0.57998593
Iteration 13, loss = 0.57477891
Iteration 14, loss = 0.56918322
Iteration 15, loss = 0.59282682
Iteration 16, loss = 0.57777886
Iteration 17, loss = 0.59656504
Iteration 18, loss = 0.56783201
Iteration 19, loss = 0.58154414
Iteration 20, loss = 0.54958555
Iteration 21, loss = 0.57545229
Iteration 22, loss = 0.56167313
Iteration 23, loss = 0.54999948
Iteration 24, loss = 0.60778798
Iteration 25, loss = 0.55960091
Iteration 26, loss = 0.57992894
Iteration 27, loss = 0.55377931
Iteration 28, loss = 0.57414194
Iteration 29, loss = 0.54391169
Iteration 30, loss = 0.53433750
Iteration 31, loss = 0.54781450
Iteration 32, los

In [13]:
confusion_matrix(ytest,ypred)

array([[10,  0,  0],
       [ 0,  9,  0],
       [ 0,  0, 11]], dtype=int64)

In [14]:
model.predict(neg_Xtrain)

array([2, 1, 2, 1, 2, 2, 2, 0, 1, 2, 1, 2, 1, 2, 1, 2, 2, 0, 2, 2, 1, 2,
       1, 0, 1, 2, 2, 1, 2, 2, 1, 2, 0, 1, 0, 2, 1, 0, 0, 1, 2, 1, 1, 0,
       1, 1, 1, 2, 0, 0, 2, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1,
       1, 0, 1, 1, 2, 1, 2, 2, 1, 0, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2,
       1, 2, 2, 0, 1, 1, 1, 2, 1, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 1, 2, 0,
       1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 0, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2,
       1, 1, 2, 0, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1,
       0, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2])

In [15]:
neg_ytrain

65     0
146    0
51     0
142    0
85     0
      ..
74     2
20     2
71     2
14     2
92     2
Length: 240, dtype: int32

In [18]:
model.predict(
    np.array([
        [4.6,3.6,1.0,0.2,1]
    ])
)

c:\Users\Jason\Desktop\git_control\ml-research\.venv\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


array([2])

In [27]:
predicted_probabilities = model.predict_proba(
    np.array([
        [4.6,3.6,1.0,0.2,1],#Negation is on, the arg max class should either be 1 or 2
        [4.6,3.6,1.0,0.2,0]#Negation is off, the arg max class should be 0
    ])
)

c:\Users\Jason\Desktop\git_control\ml-research\.venv\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [28]:
predicted_probabilities

array([[4.43954791e-05, 4.98708274e-01, 5.01247331e-01],
       [9.99997764e-01, 2.23645313e-06, 3.03234967e-18]])

In [26]:
np.random.choice([0,1,2],p=[.01,.33,.66])

2

In [47]:
np.apply_along_axis(axis=1, arr=predicted_probabilities,
    func1d = lambda x: np.random.choice(len(x), size=1, p=x)
)

array([[2],
       [0]])